In [33]:
import numpy as np
import pandas as pd
import csv
import openpyxl
import math
'''train = np.array([
    [0,0,1,2,3],
    [1,1,2,3,4],
    [2,2,3,4,5],
    [3,3,4,5,6]
]
)'''
'''
train = np.array([
    [0,1,1,1,3],
    [1,1,1,1,4],
    [2,1,1,1,5],
    [3,1,1,1,6]
]
)'''
train = pd.read_csv('train.csv').as_matrix()#read train.csv file
#print(type(train))
#xtrain=train[:,1:4]
#xtrain = xtrain/5
#true_label=train[:,4:5]
#print((xtrain),(true_label))


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [34]:
def encode_into_prob(x,max_val):
    len=x.size
    y_list=[]
    for i in range(len):
        temp=[]
        for j in range(max_val):
            if(x[i]==j):
                temp.append(1)
            else:
                temp.append(0)
            #print(temp)
        y_list.append(temp)
    return y_list
        

In [35]:
xtrain=train[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = train[:,785:]
y_true=np.array(encode_into_prob(true_label,10))
y_true=y_true.transpose()

#print(type(y_true))
#print(y_true.transpose())
print(xtrain)
print((y_true))

[[-0.55945675 -0.55945675  2.06254185 ...  2.0522595  -0.55945675
  -0.55945675]
 [-0.5380259  -0.5380259  -0.5380259  ... -0.5380259  -0.5380259
  -0.5380259 ]
 [-0.6486501  -0.6486501  -0.6486501  ... -0.6486501  -0.6486501
  -0.6486501 ]
 ...
 [-0.64973918 -0.64973918 -0.64973918 ... -0.64973918 -0.64973918
  -0.64973918]
 [-0.59745866 -0.59745866 -0.59745866 ... -0.59745866 -0.59745866
  -0.59745866]
 [-0.62629522 -0.62629522 -0.62629522 ... -0.62629522 -0.62629522
  -0.62629522]]
[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]]


In [36]:
def feedforward(batch_train,hidden_layers):
     
    global weight_matrix_list
    global bias_list
    global preactivation_list 
    preactivation_list=[]
    global activation_list
    activation_list=[]
    global y_hat
    y_hat=[]
    a_new=[]
    a_last_layer=[]
    nos_data = len(batch_train)
    #print(nos_data)
    h=batch_train.transpose()
    preactivation_list.append(h)
    activation_list.append(h)
    #print(h)
    
#     print("y_hat in feed \n",y_hat )
    
    for j in range(hidden_layers):
        
#         print(".............................")
#         print("activation for layer ", j)
#         print(activation_list[j] , activation_list[j].shape)
#         print("weight",j+1)
#         print(weight_matrix_list[j], weight_matrix_list[j].shape)
#         print("bias ",j+1)
#         print(bias_list[j],bias_list[j].shape)
        
        a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
        preactivation_list.append(a_new)
        
#         print("preactivation for layer",j+1)
#         print(preactivation_list[j+1], preactivation_list[j+1].shape)
        
        h = 1.0/(1.0+np.exp(-1*preactivation_list[j+1]))
        activation_list.append(h)

    
    j=hidden_layers
#     print(".............................")
#     print("activation for layer ", j)
#     print(activation_list[j] , activation_list[j].shape)
#     print("weight",j+1)
#     print(weight_matrix_list[j], weight_matrix_list[j].shape)
#     print("bias ",j+1)
#     print(bias_list[j],bias_list[j].shape)
    
    a_last_layer = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),activation_list[j])
    preactivation_list.append(a_last_layer)
    
#     print("last activation")
#     print(a_last_layer,a_last_layer.shape)

    y_hat = np.array(output(a_last_layer,batch_train))
    y_hat = y_hat.transpose()
    
#     print("y_hat in feedforward end")
#     print(y_hat)
    
    
def output(a_last_layer,batch_train):
    y_hat_initial=[]
    y_hat_local=[]
    nos_data=len(batch_train)
    for i in range(nos_data):
        y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

    length = len(y_hat_initial)
    for i in range(length):
        y_hat_local.append(y_hat_initial[i][0])
    
#     print("y_hat_local", y_hat_local)
#     print("y_hat_initial",y_hat_initial)
    return y_hat_local

def softmax(row_vector):
    sum_all=0
    row_vector = np.exp(row_vector)
    sum_all = np.sum(row_vector , axis=1)         #print("row_vector", row_vector)
    row_vector = (row_vector*1.0/sum_all)         #print("sum_all",sum_all)
    return row_vector

In [37]:
def backpropagation(y_true_batch):
    
#     print("weigt in backpropogation", weight_matrix_list)
#     print("...........y_true_batch..........\n",y_true_batch)
    
    global grad_a_k_list
    grad_a_k_list=[]
    
    global grad_w_k_list
    grad_w_k_list=[]
    
    global grad_h_k_list
    grad_h_k_list=[]
    
    global grad_b_k_list
    grad_b_k_list=[]
    
    global grad_b
    grad_b=[]
    
    global grad_g_dash_a_k_list
    grad_g_dash_a_k_list = []
    
    global grad_a_last
    grad_a_last=[]
    
    k = num_hidden+1
    
#     print("--------------------------in Backpropagation---------------------")
    
    grad_g_dash_a_k_list = g_dash(preactivation_list)
    
#     print("preactivation_list....................")
#     print("Nos. of matrices in preactivation_list is", len(preactivation_list))
#     print(preactivation_list)
#     print("g_dash..................")
#     print("Nos. of matrices in g_dash is ", len(grad_g_dash_a_k_list))
#     print(grad_g_dash_a_k_list)
    
    grad_a_last = -np.subtract(y_true_batch,y_hat)
    
#     print("y_true_batch \n", y_true_batch)
#     print("y_hat \n",y_hat)
    
    grad_a_k_list.insert(0,grad_a_last)
    
#     print("grad_a_last_layer", k)
#     print(grad_a_k_list[0],grad_a_k_list[0].shape)
  
    
    while(k>0):
        
#         print(".................grad_compute at layer",k)
        
        grad_w_k_list.insert(0,np.matmul(grad_a_k_list[0],activation_list[k-1].transpose()))
        
#         print("grad_a", k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print('preactivation  a_',k-1)
#         print(preactivation_list[k-1],preactivation_list[k-1].shape)
#         print("grad_w_",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
        
        grad_b_k_list.insert(0,grad_a_k_list[0])
        
#         print("grad_b",k)
#         print(grad_b_k_list[0],grad_b_k_list[0].shape)
        
        grad_h_k_list.insert(0,np.matmul(weight_matrix_list[k-1],grad_a_k_list[0]))
        
#         print("grad_w",k)
#         print(grad_w_k_list[0],grad_w_k_list[0].shape)
#         print("grad_a",k)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
#         print("grad_h",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
        
        grad_a_k_list.insert(0,np.multiply(grad_h_k_list[0],grad_g_dash_a_k_list[k-1]))
        
#         print("grad_h_k_list[0]",k-1)
#         print(grad_h_k_list[0],grad_h_k_list[0].shape)
#         print("grad_g_dash_a_k_list[k-1]",k-1)
#         print(grad_g_dash_a_k_list[k-1],grad_g_dash_a_k_list[k-1].shape)
#         print("grad_a",k-1)
#         print(grad_a_k_list[0],grad_a_k_list[0].shape)
        k=k-1
#         print("grad_a_ik_list",grad_a_k_list[0])
    
    sum_all_grad_b_of_data(grad_b_k_list)
    

def sum_all_grad_b_of_data(grad_b_k_list):
    length=len(grad_b_k_list)
    i=0
    while(i<length):
        row=[]
        row = np.sum(grad_b_k_list[i],1)
        j = len(row)
        #print(row.shape)
        row = row[:, np.newaxis]
        #print(row.shape , row)
        grad_b.append(row)
        i+=1
    #print("grad_b",grad_b)

def g_dash(preactivation_list):
    length=len(preactivation_list)
    a_modified = preactivation_list[0:length]
    g_dash_a_k_list=[]
    for i in range(length):
        a=(1.0/(1+np.exp(-a_modified[i])))*(1-(1.0/(1+np.exp(-a_modified[i]))))
        g_dash_a_k_list.append(a)
    return g_dash_a_k_list


In [38]:
def decode_yhat_to_classes(y_hat_local):
    col = len(y_hat_local[0])
    max_val=0
    for j in range(col):
        max_val = y_hat_local[0][j]
        index=0
        for i in range(9):
            if(y_hat_local[i+1][j] > max_val):
                y_hat_local[index][j]=0
                index = i+1
                max_val=y_hat[i+1][j]
            else :
                y_hat_local[i+1][j]=0
        y_hat_local[index][j]=1
    return y_hat_local

In [39]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i+1)
                break
    return np.array(y_predict)

In [40]:
#ip_neurons = 3 #input layer
ip_neurons = 784
num_hidden = 2
encoding_bits=10
sizes = [50,100]
sizes.insert(0,ip_neurons)
sizes.append(encoding_bits)
print(sizes)
print(len(sizes))

weight_matrix_list = []

bias_list =[]
y_hat= []

for i in range(int(num_hidden)+1):
    weight_matrix_list.append(np.random.rand(sizes[i],sizes[i+1])/np.sqrt(sizes[i]))
    bias_list.append(0*np.random.rand(sizes[i+1],1))
print(weight_matrix_list)
#print(len(weight_matrix_list))

print(bias_list)
#print(hlist)



[784, 50, 100, 10]
4
[array([[0.00109635, 0.03219259, 0.01074146, ..., 0.00643851, 0.02192284,
        0.03315315],
       [0.00630526, 0.00129497, 0.0117237 , ..., 0.03284712, 0.01340493,
        0.01280288],
       [0.01287493, 0.01692015, 0.02666916, ..., 0.01502726, 0.03501396,
        0.00380961],
       ...,
       [0.01356819, 0.03011855, 0.03057457, ..., 0.02199732, 0.03455368,
        0.01514046],
       [0.01647585, 0.02579763, 0.01382885, ..., 0.00979694, 0.02012946,
        0.03198862],
       [0.00777396, 0.008059  , 0.02972664, ..., 0.01440984, 0.00853415,
        0.0058198 ]]), array([[0.09876103, 0.03416655, 0.01274727, ..., 0.11155943, 0.11013763,
        0.01382684],
       [0.04834662, 0.06988934, 0.12951638, ..., 0.09927345, 0.12455441,
        0.07751043],
       [0.14055496, 0.10900656, 0.00912434, ..., 0.00400206, 0.01454718,
        0.09377716],
       ...,
       [0.115821  , 0.02716956, 0.0829549 , ..., 0.03346369, 0.13894221,
        0.14034268],
       [0.00

In [41]:
def grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    
    len_w = len(weight_matrix_list)
    len_b = len(bias_list)
    data_length = len(xtrain)
    batch_size = 10
    eta=0.01
    no_of_batch = int(data_length/batch_size)
    epochs =30
    steps = 0
    loss=0
    error = 0
    nos_points = 0
    file = open('log_file.txt', 'w')
    for t in range(epochs):
        loss=0
        error=0
        for batch in range(no_of_batch):
            dw=[]
            db=[]
            for i in range(int(num_hidden)+1):
                dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
                db.append(0*np.random.rand(sizes[i+1],1))
            
            for data_index_in_batch in range(batch_size):
                l=batch_size*batch + data_index_in_batch
                r=batch_size*batch + data_index_in_batch + 1
#                 print("before\n",data_index_in_batch)
                feedforward(xtrain[l:r,:],num_hidden)
#                 print("after\n",y_hat)
#                 print(l , r)
                backpropagation(y_true[:,l:r])

                for j in range(len_w):
                    dw[j] += grad_w_k_list[j].transpose()

                for j in range(len_b):
                    db[j] += grad_b[j]
                
                
                loss += cal_loss(y_hat,y_true[:,l:r])
                
#                 error += cal_error(y_hat,y_true[:,l:r])
                
#                 nos_points +=1
                
            for j in range(len_w):
                weight_matrix_list[j] = weight_matrix_list[j] - eta*dw[j]

            for j in range(len_b):
                bias_list[j] =  bias_list[j] - eta*db[j]
            
            if (steps%300==0):
                file.write("Epoch "+str(t+1)+", Step "+str(steps)+", loss : "+str(loss)+", error : "+str(error/nos_points)+", lr: " + str(eta)+ "\n")
#                 print("loss \n",loss)
            steps += 1
            
            
#             print("after batch ", batch)
#             print("y_hat", y_hat)
            
#             print("subtracted weight\n", weight_matrix_list)
            
            
            
        print("end  epochs :  ", t, loss)
#     file.close()

In [42]:
def cal_loss(y_hat,y_true):
    loss = 0
    for i in range(10):
        if(y_hat[i]!=0):
            loss += (-1)*y_true[i]*math.log(y_hat[i]) 
    
    return loss
        
def cal_error(y_hat,y_true):
    true = classifcation(y_true)
    predicted = classifcation(decode_yhat_to_classes(y_hat))
    if (true != predicted):
        return 1
    return 0

In [43]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)

In [29]:
grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes)
#print(y_hat)0
#print(len(xtrain))

end  epochs :   0 [82314.40653052]
end  epochs :   1 [29755.57730271]
end  epochs :   2 [20758.5163485]
end  epochs :   3 [15594.96785681]
end  epochs :   4 [11860.51197763]
end  epochs :   5 [9117.23096232]
end  epochs :   6 [6936.12526568]
end  epochs :   7 [5459.28412708]
end  epochs :   8 [4338.13116284]
end  epochs :   9 [3671.32076224]
end  epochs :   10 [3024.51611821]
end  epochs :   11 [2444.70391476]
end  epochs :   12 [1916.41267753]
end  epochs :   13 [1512.8028853]
end  epochs :   14 [1190.46374851]
end  epochs :   15 [953.02689759]
end  epochs :   16 [805.33976093]
end  epochs :   17 [700.92753076]
end  epochs :   18 [629.48692945]
end  epochs :   19 [564.74146316]
end  epochs :   20 [509.3282782]
end  epochs :   21 [461.85418712]
end  epochs :   22 [421.28061304]
end  epochs :   23 [385.38913952]
end  epochs :   24 [358.88395356]
end  epochs :   25 [331.71697235]
end  epochs :   26 [309.53481716]
end  epochs :   27 [287.96988986]
end  epochs :   28 [267.21883469]
end  ep

In [30]:
feedforward(xtrain,num_hidden)
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[2.64353364e-13 3.73949893e-16 4.19549592e-08 ... 9.59643030e-11
  4.35510604e-16 8.83201612e-18]
 [4.08770298e-03 3.39618661e-16 9.99999038e-01 ... 1.62769773e-08
  6.01440135e-11 1.18966762e-17]
 [8.05967789e-19 9.72224838e-09 2.03468069e-07 ... 1.54305413e-04
  2.91701026e-08 6.46758407e-10]
 ...
 [6.97302944e-09 4.71779472e-09 3.63153787e-07 ... 1.95532211e-05
  5.13815767e-18 5.86690192e-12]
 [1.94440597e-13 3.21956679e-10 2.03199742e-15 ... 1.03702674e-08
  1.14170241e-16 5.59661210e-07]
 [5.13473003e-12 4.28922237e-06 3.34932279e-07 ... 4.71790691e-03
  9.99998552e-01 1.07287603e-11]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


In [31]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))
print(true_label)

[4 6 1 ... 6 9 6]
[[4]
 [6]
 [1]
 ...
 [6]
 [9]
 [6]]
[[4.]
 [6.]
 [1.]
 ...
 [6.]
 [9.]
 [6.]]


In [32]:
cnt=0
for i in range(55000):
    if(y_predict[i]==true_label[i]):
        cnt+=1
acc = cnt/55000
print(acc)

0.9993454545454545


In [82]:
valid = pd.read_csv('valid.csv').as_matrix()
xtrain=valid[:,1:785]

xmean = np.mean(xtrain , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
xtrain = xtrain - means_expanded

xstd = np.std(xtrain , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
xtrain = xtrain*1.0/std_expanded 

true_label = valid[:,785:]


C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [83]:
feedforward(xtrain,num_hidden)
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[4.10191211e-04 9.84131298e-06 2.83954028e-05 ... 8.20260749e-01
  1.11356195e-02 1.68644264e-04]
 [9.59198810e-01 7.04160543e-04 3.83927059e-03 ... 1.76400583e-02
  1.23599433e-02 5.09703830e-04]
 [8.98744705e-03 8.30065539e-03 5.05046872e-06 ... 3.03206052e-02
  2.54696153e-04 9.94040979e-01]
 ...
 [2.80833030e-03 8.96574668e-04 4.74751884e-04 ... 1.08982260e-04
  3.93649122e-01 3.69091508e-04]
 [5.01367429e-03 2.47946214e-04 6.24869263e-04 ... 8.18720513e-05
  1.21391096e-01 4.50436386e-03]
 [1.75800697e-07 9.83015684e-01 1.69811286e-06 ... 2.78270759e-07
  3.59006330e-07 1.26022428e-04]]
[[0. 0. 0. ... 1. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 1.]
 ...
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [84]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))
print(true_label)

[1 9 4 ... 0 7 2]
[[1]
 [9]
 [4]
 ...
 [0]
 [7]
 [2]]
[[1.]
 [9.]
 [4.]
 ...
 [0.]
 [6.]
 [2.]]


In [86]:
cnt=0
k = len(true_label)
for i in range(k):
    if(y_predict[i]==true_label[i]):
        cnt+=1
acc = cnt/k
print(acc)

0.7706


In [79]:

xtest = pd.read_csv('test.csv').as_matrix()
test=xtest[:,1:]

xmean = np.mean(test , axis=1)
means_expanded = np.outer(xmean, np.ones(784))
#print(means_expanded)
test = test - means_expanded

xstd = np.std(test , axis=1)
std_expanded = np.outer(xstd, np.ones(784))
#print(std_expanded)
test = test*1.0/std_expanded 

feedforward(test,num_hidden)
#print(y_hat)
# y_hat = decode_yhat_to_classes(y_hat)
#print(y_hat)

C:\Users\LN Pandey\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


In [80]:
print(y_hat)
y_hat = decode_yhat_to_classes(y_hat)
print(y_hat)

[[2.71660489e-04 5.18170676e-02 4.55925136e-02 ... 1.19666898e-03
  3.26600394e-04 1.85743674e-05]
 [2.15264564e-04 6.06552503e-04 3.92482707e-01 ... 4.56937506e-02
  5.33260342e-05 9.96855782e-01]
 [1.02376299e-01 1.41460633e-01 6.32031913e-02 ... 8.54159541e-03
  3.72155543e-03 1.66344453e-04]
 ...
 [4.03617587e-03 1.11531948e-01 2.94184984e-02 ... 3.01454251e-05
  6.90161724e-02 2.02402223e-03]
 [1.00427913e-02 1.39394461e-01 7.13397059e-04 ... 5.54354918e-07
  9.00542336e-01 8.15958935e-07]
 [4.39663052e-02 6.22723590e-05 3.45196609e-01 ... 9.30603883e-01
  5.57629853e-05 1.71033985e-05]]
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 1. 0. 0.]]


In [81]:
def classifcation(y_hat):
    y_predict=[]
    col = len(y_hat[0])
    for j in range(col):
        for i in range(10):
            if(y_hat[i][j]==1):
                y_predict.append(i)
                break
    return np.array(y_predict)

In [54]:
y_predict = classifcation(y_hat)
print((y_predict))
y_predict = y_predict[:, np.newaxis]
print((y_predict))

[6 6 1 ... 9 8 1]
[[6]
 [6]
 [1]
 ...
 [9]
 [8]
 [1]]


In [55]:
def add_id_to_y_predict(y_predict):
    ide=[]
    row=len(y_predict)
    for i in range(row):
        ide.append(i)
    ide=np.array(ide)
    ide=ide[:,np.newaxis]
    y= np.hstack((ide,y_predict))
    #print(y_predict, ide)
    return y           

In [56]:
print(y_predict)
#y_predict= np.random.randint(10, size=(10000, 1))
y_predict= add_id_to_y_predict(y_predict)
print(y_predict)
x,y = y_predict.shape
#print(x,y)

[[6]
 [6]
 [1]
 ...
 [9]
 [8]
 [1]]
[[   0    6]
 [   1    6]
 [   2    1]
 ...
 [9997    9]
 [9998    8]
 [9999    1]]


In [57]:
with open('predict_vanilla_grad_descent2.csv', 'w', newline='') as f:
    thewriter = csv.writer(f)
    row,col=y_predict.shape 
    thewriter.writerow(['id','label'])
    for i in range(row):
        thewriter.writerow(y_predict[i])
#out.close()
#np.savetxt("foo.csv", y_predict, delimiter=",")

[[5.93764916e-15 4.05568165e-07 3.59781929e-14 ... 3.89901106e-10
  7.59965544e-31 3.19362134e-15]
 [9.94611843e-01 2.68611797e-08 9.99999998e-01 ... 4.98472730e-10
  7.70579926e-20 1.56223730e-04]
 [1.67894717e-12 6.30483232e-05 2.30425111e-09 ... 1.83559673e-08
  8.51176117e-18 7.86255050e-09]
 ...
 [1.99193705e-09 3.19231017e-01 1.09848738e-15 ... 4.23982938e-08
  9.55262076e-34 1.56278549e-04]
 [2.15413602e-17 7.96073113e-07 1.65730063e-20 ... 6.59005464e-18
  2.67756736e-38 4.73634749e-04]
 [2.08866479e-21 1.35450757e-01 2.80597335e-19 ... 6.99727562e-05
  1.00000000e+00 7.00380706e-04]]
[[0. 0. 0. ... 0. 0. 0.]
 [1. 0. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]]


[1 5 1 ... 6 9 6]
[[1]
 [5]
 [1]
 ...
 [6]
 [9]
 [6]]
[[4.]
 [6.]
 [1.]
 ...
 [6.]
 [9.]
 [6.]]


In [25]:
cnt=0
for i in range(55000):
    if(y_predict[i]==true_label[i]):
        cnt+=1
acc = cnt/55000
print(acc)

0.7410363636363636


In [127]:
def adam(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
    
    m_w = [] 
    m_b = []
    for i in range(int(num_hidden)+1):
        m_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        m_b.append(0*np.random.rand(sizes[i+1],1))
    
    v_w = [] 
    v_b =[]
    for i in range(int(num_hidden)+1):
        v_w.append(0*np.random.rand(sizes[i],sizes[i+1]))
        v_b.append(0*np.random.rand(sizes[i+1],1))
        
    len_w = len(weight_matrix_list)  
    len_b = len(bias_list)
    
    eta = 0.0001
    batch_size = 10
    num_points_seen = 0
    beta1 , beta2 , eps = 0.9 , 0.999 , 1e-8
    epochs = 5
    t=0
    data_length = len(xtrain)
    no_of_batch = int(data_length/batch_size)
    
    for t in range(epochs):
        loss=0
        for batch in range(no_of_batch):
            dw=[]
            db=[]
            for i in range(int(num_hidden)+1):
                dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
                db.append(0*np.random.rand(sizes[i+1],1))
            
            for i in range(batch_size):
                l = batch_size*batch + i
                r = batch_size*batch + i + 1
                feedforward(xtrain[l:r,:],num_hidden)
                #print(l , r)
                backpropagation(y_true[:,l:r])

                for j in range(len_w): 
                    dw[j] += grad_w_k_list[j].transpose()
                                    
                for j in range(len_b):
                    db[j] += grad_b[j]
                
                loss += cal_loss(y_hat,y_true[:,l:r])
                                
            for j in range(len_w): 
                m_w[j] = beta1*m_w[j] + (1-beta1)*dw[j]
                                    
            for j in range(len_b):
                m_b[j] = beta1*m_b[j] + (1-beta1)*db[j]    
            
            for j in range(len_w): 
                v_w[j] = beta2*v_w[j] + (1-beta2)*(dw[j]**2)
                                    
            for j in range(len_b):
                v_b[j] = beta2*v_b[j] + (1-beta2)*(db[j]**2)
                
            for j in range(len_w): 
                m_w[j] = (m_w[j]*1.0)/(1.0*(1-math.pow(beta1 , batch + 1)))
                                    
            for j in range(len_b):
                m_b[j] = (m_b[j]*1.0)/(1.0*(1-math.pow(beta1 , batch + 1)))
             
            for j in range(len_w): 
                v_w[j] = (v_w[j]*1.0)/(1.0*(1-math.pow(beta2 , batch + 1)))
                                    
            for j in range(len_b):
                v_b[j] = (v_b[j]*1.0)/(1.0*(1-math.pow(beta2 , batch + 1)))
                
            for j in range(len_w):
                weight_matrix_list[j] -= (eta/1.0*np.sqrt(v_w[j] + eps))*m_w[j]
            
            for j in range(len_b):
                bias_list[j] -= (eta/1.0*np.sqrt(v_b[j] + eps))*m_b[j]
        
        print("end of epoch" , t, loss)
            
#             print("batch", batch)
#             y_hat_local = decode_yhat_to_classes(y_hat)
#             print(y_hat)
                
#         print("end  epochs :  ", t)
#         y_hat_local = decode_yhat_to_classes(y_hat)
#         print(y_hat)
#     print(bias_list)
#     print(preactivation_list)

In [ ]:
# def test_func(weight_matrix_list,bias_list,test,hidden_layers):
#     nos_data = len(test)
#     #print(nos_data)
#     #preactivation_list_new=[]
#     #activation_list_new=[]
#     #for i in range(nos_data):
#     h = test.transpose()
#     #preactivation_list_new.append(h)
#     #activation_list_new.append(h)
#     #print(h)
#     for j in range(hidden_layers):
#         a_new = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),h)
#         #preactivation_list_new.append(a_new)
#         h = 1.0/(1.0+np.exp(-a_new))
#         #activation_list_new.append(h)
#         #print("next activation", h)

#     j=hidden_layers
#     #hlist.append(h)
#     a_last = bias_list[j]+np.matmul(weight_matrix_list[j].transpose(),h)
#     #preactivation_list.append(a_last)
#     print("last activation", a_last)
#     #y_hat_new = np.array(output(a_last))
#     #y_hat_new = y_hat_new.transpose()
#     #print("y_hat", y_hat)
#     #print("y_hat_global", y_hat_global)
#     #print(y_hat.transpose())
#     return y_hat

In [ ]:
# test = np.array([
#     [5,6,7],
#     [6,7,8]
# ]
# )
# y=test_func(weight_matrix_list,bias_list,test,num_hidden)
# print(y)

In [ ]:
# x= np.array([
#     [0,0,1,2,3],
#     [1,1,2,3,4],
#     [2,2,3,4,5],
#     [3,3,4,5,6]
# ]
# )
# y = np.array([
#     [0,0,1,2,3],
#     [1,1,2,3,4],
#     [2,2,3,4,5],
#     [3,3,4,5,6]
# ]
# )
# print(np.subtract(x,y))
# print(type(x))

In [ ]:
# s=np.tile(np.arange(1,10,10),(10,4))
# print(s)
# s[:,i:i+1]

In [ ]:
# s=np.tile(np.arange(1,10,10),(10,4))
# print(s)
# print(s[:,i:i+1])
# p=softmax(s[:,i:i+1].transpose())
# print("P", p)

In [190]:
# x = np.array([
#     [0,0,1,2,3],
#     [1,1,2,3,4],
#     [2,2,3,4,5],
#     [3,3,4,5,6]
# ])
# #print(type(x))
# y = np.mean(x , axis=1)
# means_expanded = np.outer(y, np.ones(5))
# print(means_expanded)
# z=x-means_expanded
# a = np.std(x,axis=1)
# variance_extended = np.outer(a,np.ones(5))
# print(a)
# print(variance_extended)
# unit_x= x/variance_extended
# print(unit_x)
# check = np.std(unit_x, axis=1)
# print(check)

In [191]:
# mat = np.array([
#     [0,0,1,2,7],
#     [1,1,2,3,4],
#     [2,2,3,4,5],
#     [3,3,4,5,6]
# ])
# sum_col = np.sum(mat[0:1,:],axis=1)
# print(sum_col)

In [ ]:
# def output(a_last_layer,batch_train):
#     y_hat_initial=[]
#     y_hat_local=[]
#     nos_data=len(batch_train)
#     for i in range(nos_data):
#         y_hat_initial.append(softmax(a_last_layer[:,i:i+1].transpose()))

#     length = len(y_hat_initial)
#     for i in range(length):
#         y_hat_local.append(y_hat_initial[i][0])
    
#     print("y_hat_local", y_hat_local)
#     print("y_hat_initial",y_hat_initial)
#     return y_hat_local

In [ ]:
# def softmax(row_vector):
#     i = row_vector.size
#     sum_all=0
#     row_vector = np.exp(row_vector)
#     for j in range(i):
#         sum_all+= row_vector[0][j]
#     row_vector = (row_vector*1.0/sum_all)
#     return row_vector

In [ ]:
# def grad_descent(weight_matrix_list,bias_list,xtrain,num_hidden,sizes):
#     t=0
#     max_iteration=10
    
#     eta=0.01
#     data_length = len(xtrain)
#     batch_size = 10
#     #batch_size = data_length
#     no_of_batch = int(data_length/batch_size)
#     #no_of_batch =1
#     while(t < max_iteration):
#         dw=[]
#         db=[]
        
#         for i in range(int(num_hidden)+1):
#             dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
#             db.append(0*np.random.rand(sizes[i+1],1))
            
        
        
#         for batch in range(no_of_batch):
#             l=batch_size*batch
#             r=batch_size*batch+batch_size
#             feedforward(xtrain[l:r,:],num_hidden)
#             backpropagation(y_true[:,l:r])
            
#             length=len(weight_matrix_list)
#             i=0
#             #print("grad_w",grad_w_k_list)
#             #print("a_matrix",preactivation_list)
#             #print("hactivation",activation_list)
#             #print("hat",y_hat)
            
#             while(i<length):
#                 dw[i] += grad_w_k_list[i].transpose()
#                 i+=1
#                 #print("weight_matrix_list", weight_matrix_list[i])
#                 #print("grad_w_k_list", grad_w_k_list[i])
#                 #print("weight_matrix_list", weight_matrix_list[i])
                
#             length=len(bias_list)
#             i=0
            
#             while(i<length):
#                 db[i] += grad_b[i]
#                 i+=1
#                 #print("y_hat", y_hat)
#                 #y_hat_local = decode_yhat_to_classes(y_hat)
#                 #if(batch == batch_size-1):
#                 #print("y_hat", y_hat)
#                 #print("y_hat for batch ", batch , y_hat_local)
        
#         length=len(weight_matrix_list)
#         i=0
#         while(i<length):
#             weight_matrix_list[i] -= eta*dw[i]
#             i+=1
        
#         length=len(bias_list)
#         i=0        
#         while(i<length):
#             bias_list[i] -= eta*db[i]
#             i+=1
#         t+=1
#         print("end  epochs :  ", t)
#         #print(y_hat)
# #     print(bias_list)
# #     print(preactivation_list)

In [ ]:
# dw=[]
# db=[]
        
# for i in range(int(num_hidden)+1):
#     dw.append(0*np.random.rand(sizes[i],sizes[i+1]))
#     db.append(0*np.random.rand(sizes[i+1],1))
# print(dw)
# print(db)

In [ ]:
# def decode_yhat_to_classes(y_hat):
#     col = len(y_hat[0])
#     max_val=0
#     for j in range(col):
#         max_val = y_hat[0][j]
#         index=0
#         for i in range(9):
#             if(y_hat[i+1][j] > max_val):
#                 y_hat[index][j]=0
#                 index = i+1
#                 max_val=y_hat[i+1][j]
#             else :
#                 y_hat[i+1][j]=0
#         y_hat[index][j]=1
#     return y_hat